In [4]:
import os
import json
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas.io.json import json_normalize
import time
from datetime import datetime

In [5]:
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [6]:
import lightgbm as lgb
from catboost import CatBoostRegressor
%matplotlib inline
pd.options.display.max_columns = 100

In [10]:
CatBoostRegressor??

In [36]:
JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
train = pd.read_csv('train.csv',converters={column: json.loads for column in JSON_COLUMNS},
                    dtype={'fullVisitorId': 'str'})
test = pd.read_csv('test.csv',converters={column: json.loads for column in JSON_COLUMNS}, 
                    dtype={'fullVisitorId': 'str'})

In [65]:
train.shape,test.shape

((903653, 32), (804684, 32))

In [66]:
train.columns

Index(['channelGrouping', 'fullVisitorId', 'visitNumber', 'device.browser',
       'device.deviceCategory', 'device.isMobile', 'device.operatingSystem',
       'geoNetwork.city', 'geoNetwork.continent', 'geoNetwork.country',
       'geoNetwork.metro', 'geoNetwork.networkDomain', 'geoNetwork.region',
       'geoNetwork.subContinent', 'totals.hits', 'totals.pageviews',
       'trafficSource.adContent',
       'trafficSource.adwordsClickInfo.adNetworkType',
       'trafficSource.adwordsClickInfo.gclId',
       'trafficSource.adwordsClickInfo.page',
       'trafficSource.adwordsClickInfo.slot', 'trafficSource.campaign',
       'trafficSource.keyword', 'trafficSource.medium',
       'trafficSource.referralPath', 'trafficSource.source',
       'diff_visitId_time', 'WoY', 'month', 'quarter_month', 'weekday',
       'visit_hour'],
      dtype='object')

In [71]:
#train.describe()#train.infer_objects()

In [73]:
print(train.dtypes)

channelGrouping                                  int64
fullVisitorId                                   object
visitNumber                                      int64
device.browser                                   int64
device.deviceCategory                            int64
device.isMobile                                  int64
device.operatingSystem                           int64
geoNetwork.city                                  int64
geoNetwork.continent                             int64
geoNetwork.country                               int64
geoNetwork.metro                                 int64
geoNetwork.networkDomain                         int64
geoNetwork.region                                int64
geoNetwork.subContinent                          int64
totals.hits                                      int64
totals.pageviews                                 int64
trafficSource.adContent                          int64
trafficSource.adwordsClickInfo.adNetworkType     int64
trafficSou

In [84]:
train.isnull().values.any(),train.isnull().values.sum()

(False, 0)

In [22]:
#train['totals'] = train['totals'].apply(lambda x:eval(x))
#transactions = [list(x.keys()) for x in train['totals']]
#list1 = ['transactionRevenue' in x for x in transactions]
#list1.count(True)

In [37]:
JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
def df_normalize(df):
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    return df

train = df_normalize(train)
test = df_normalize(test)

train[(train['totals.transactionRevenue'].notnull())].head()

In [50]:
[1,2]+[7,8], type(test.columns[0])

([1, 2, 7, 8], str)

In [54]:
def process_dfs(train, test):
    columns = [col for col in test.columns if train[col].nunique() > 1]
    train = train[columns + ['totals.transactionRevenue']]
    test = test[columns]
    test['totals.transactionRevenue'] = np.nan

    trn_len = train.shape[0]
    merged_df = pd.concat([train, test])

    merged_df['diff_visitId_time'] = merged_df['visitId'] - merged_df['visitStartTime']
    merged_df['diff_visitId_time'] = (merged_df['diff_visitId_time'] != 0).astype(int)
    
    
    format_str = '%Y%m%d' 
    merged_df['formated_date'] = merged_df['date'].apply(lambda x: datetime.strptime(str(x), format_str))
    merged_df['WoY'] = merged_df['formated_date'].apply(lambda x: x.isocalendar()[1])
    merged_df['month'] = merged_df['formated_date'].apply(lambda x:x.month)
    merged_df['quarter_month'] = merged_df['formated_date'].apply(lambda x:x.day//8)
    merged_df['weekday'] = merged_df['formated_date'].apply(lambda x:x.weekday())
    merged_df['formated_visitStartTime'] = merged_df['visitStartTime'].apply(
        lambda x: time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(x)))
    merged_df['formated_visitStartTime'] = pd.to_datetime(merged_df['formated_visitStartTime'])
    merged_df['visit_hour'] = merged_df['formated_visitStartTime'].apply(lambda x: x.hour)

    del merged_df['visitStartTime']
    del merged_df['formated_visitStartTime']
    del merged_df['visitId']
    del merged_df['sessionId']
    del merged_df['date']
    del merged_df['formated_date']
    
    for col in merged_df.columns:
        if col in ['fullVisitorId', 'month', 'quarter_month', 'weekday', 'visit_hour', 'WoY']: continue
        if merged_df[col].dtypes == object or merged_df[col].dtypes == bool:
            merged_df[col], indexer = pd.factorize(merged_df[col])

    
    train = merged_df[:trn_len]
    test = merged_df[trn_len:]
    return train, test


In [55]:
%%time
train, test = process_dfs(train, test)

C:\Users\Neha\Anaconda2\envs\Tensorflow\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [56]:
target = train['totals.transactionRevenue'].fillna(0).astype(float)
target = target.apply(lambda x: np.log1p(x))
del train['totals.transactionRevenue'],test['totals.transactionRevenue']

In [62]:
test.head()

,channelGrouping,fullVisitorId,visitNumber,device.browser,device.deviceCategory,device.isMobile,device.operatingSystem,geoNetwork.city,geoNetwork.continent,geoNetwork.country,...,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source,diff_visitId_time,WoY,month,quarter_month,weekday,visit_hour
0,0,6167871330617112363,2,0,0,0,1,8,0,13,...,0,0,-1,0,0,42,10,2,0,3
1,0,0643697640977915618,1,0,0,0,0,232,2,2,...,0,0,-1,0,0,42,10,2,0,10
2,0,6059383810968229466,1,0,0,0,1,1,2,10,...,0,0,-1,0,0,42,10,2,0,1
3,0,2376720078563423631,1,4,1,1,4,23,3,33,...,0,0,-1,0,0,42,10,2,0,15
4,0,2314544520795440038,1,4,0,0,1,49,3,33,...,0,0,-1,0,0,42,10,2,0,22


In [63]:
def rmse(y_true, y_pred):
    return round(np.sqrt(mean_squared_error(y_true, y_pred)), 5)

In [ ]:
def load_preprocessed_dfs(drop_full_visitor_id=True):
    """
    Loads files `TRAIN`, `TEST` and `Y` generated by preprocess() into variables
    """
    X_train = pd.read_csv(TRAIN, converters={'fullVisitorId': str})
    X_test = pd.read_csv(TEST, converters={'fullVisitorId': str})
    y_train = pd.read_csv(Y, names=['LogRevenue']).T.squeeze()
    
    # This is the only `object` column, we drop it for train and evaluation
    if drop_full_visitor_id: 
        X_train = X_train.drop(['fullVisitorId'], axis=1)
        X_test = X_test.drop(['fullVisitorId'], axis=1)
    return X_train, y_train, X_test

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, random_state=1)

print(f"Train shape: {X_train.shape}")
print(f"Validation shape: {X_val.shape}")
print(f"Test (submit) shape: {X_test.shape}")

In [ ]:
def run_lgb(X_train, y_train, X_val, y_val, X_test):
    
    params = {
        "objective" : "regression",
        "metric" : "rmse",
        "num_leaves" : 40,
        "learning_rate" : 0.005,
        "bagging_fraction" : 0.6,
        "feature_fraction" : 0.6,
        "bagging_frequency" : 6,
        "bagging_seed" : 42,
        "verbosity" : -1,
        "seed": 42
    }
    
    lgb_train_data = lgb.Dataset(X_train, label=y_train)
    lgb_val_data = lgb.Dataset(X_val, label=y_val)

    model = lgb.train(params, lgb_train_data, 
                      num_boost_round=5000,
                      valid_sets=[lgb_train_data, lgb_val_data],
                      early_stopping_rounds=100,
                      verbose_eval=500)

    y_pred_train = model.predict(X_train, num_iteration=model.best_iteration)
    y_pred_val = model.predict(X_val, num_iteration=model.best_iteration)
    y_pred_submit = model.predict(X_test, num_iteration=model.best_iteration)

    print(f"LGBM: RMSE val: {rmse(y_val, y_pred_val)}  - RMSE train: {rmse(y_train, y_pred_train)}")
    return y_pred_submit, model

In [ ]:
%%time
# Train LGBM and generate predictions
lgb_preds, lgb_model = run_lgb(X_train, y_train, X_val, y_val, X_test)

In [ ]:
print("LightGBM features importance...")
gain = lgb_model.feature_importance('gain')
featureimp = pd.DataFrame({'feature': lgb_model.feature_name(), 
                   'split': lgb_model.feature_importance('split'), 
                   'gain': 100 * gain / gain.sum()}).sort_values('gain', ascending=False)
print(featureimp[:10])

In [ ]:
def run_catboost(X_train, y_train, X_val, y_val, X_test):
    model = CatBoostRegressor(iterations=1000,
                             learning_rate=0.05,
                             depth=10,
                             eval_metric='RMSE',
                             random_seed = 42,
                             bagging_temperature = 0.2,
                             od_type='Iter',
                             metric_period = 50,
                             od_wait=20)
    model.fit(X_train, y_train,
              eval_set=(X_val, y_val),
              use_best_model=True,
              verbose=True)
    
    y_pred_train = model.predict(X_train)
    y_pred_val = model.predict(X_val)
    y_pred_submit = model.predict(X_test)

    print(f"CatB: RMSE val: {rmse(y_val, y_pred_val)}  - RMSE train: {rmse(y_train, y_pred_train)}")
    return y_pred_submit, model

In [ ]:
%%time
# Train Catboost and generate predictions
cat_preds, cat_model = run_catboost(X_train, y_train, X_val, y_val,  X_test)

In [ ]:
ensemble_preds_70_30_00 = 0.7 * lgb_preds + 0.3 * cat_preds 
ensemble_preds_70_25_05 = 0.7 * lgb_preds + 0.25 * cat_preds 